In [1]:
import os
import pandas as pd
import pandas as pd
import json
from ydata_profiling import ProfileReport
from ydata_profiling.utils.cache import cache_file
import seaborn as sns

In [74]:
#load metadata
with open('data/processing/metadata.json', 'r') as f:
    definitions = json.load(f)

with open('data/processing/industry-recode.json', 'r') as f:
    industry_recode = json.load(f)

with open('data/processing/occupasion-recode.json', 'r') as f:
    occupasion_recode = json.load(f)

with open('data/processing/education-recode.json', 'r') as f:
    education_recoder = json.load(f)

with open('data/processing/martial-recode.json', 'r') as f:
    martial_recoder = json.load(f)

with open('data/processing/unemployment-recode.json', 'r') as f:
    unemployment_recoder = json.load(f)

with open('data/processing/typeemployment-recode.json', 'r') as f:
    typeemployment_recoder = json.load(f)

In [75]:
#load the data
df = pd.read_csv('data/raw/census_income_learn.csv', header=None, names=definitions.keys())

In [76]:
# cleaning

# remove ignore col
try: 
    df.drop(columns=['ignore'], inplace=True)
except:
    pass

# fix year"
if df['year'].min() < 100:
    df['year'] = df['year'] + 1900

# map industry code
if df['detailed_industry_recode'].dtype == 'int64':
    df['detailed_industry_recode'] = df['detailed_industry_recode'].astype(str).map(industry_recode)

# map occupation code
if df['detailed_occupation_recode'].dtype == 'int64':
    df['detailed_occupation_recode'] = df['detailed_occupation_recode'].astype(str).map(occupasion_recode)

if 'Advanced Degree' not in df['education'].unique():
    df['education'] = df['education'].map(education_recoder)

if 'Married' not in df['marital_stat'].unique():
    df['marital_stat'] = df['marital_stat'].map(martial_recoder)

if ' Other job loser' in df['reason_for_unemployment'].unique():
    df['reason_for_unemployment'] = df['reason_for_unemployment'].map(unemployment_recoder)

if ' Not in labor force' in df['full_or_part_time_employment_stat'].unique():
    df['full_or_part_time_employment_stat'] = df['full_or_part_time_employment_stat'].map(typeemployment_recoder)

# drop duplicates values
df.drop_duplicates(inplace=True)

In [81]:
df['tax_filer_stat'].unique()

array([' Nonfiler', ' Head of household', ' Joint both under 65',
       ' Single', ' Joint both 65+', ' Joint one under 65 & one 65+'],
      dtype=object)

In [73]:
df['full_or_part_time_employment_stat'].unique()

array([' Not in labor force', ' Children or Armed Forces',
       ' Full-time schedules', ' Unemployed full-time',
       ' Unemployed part- time', ' PT for non-econ reasons usually FT',
       ' PT for econ reasons usually PT',
       ' PT for econ reasons usually FT'], dtype=object)

In [53]:
df['full_or_part_time_employment_stat'].map(typeemployment_recoder).value_counts()

full_or_part_time_employment_stat
Not Employed    147252
FTE              43038
PTE               5056
Name: count, dtype: int64

In [35]:
ProfileReport(df, title = 'census data raw report').to_file('data/raw/census_data_raw_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]